Imports necessary libraries including cv2, pandas, numpy, and sklearn.

In [ ]:
import cv2
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import os
from numpy import vstack
from numpy import concatenate
from sklearn.model_selection import train_test_split

Reads a csv file named "swarmIoT_full_labeled_dataset.csv" using the pandas library and assigns the 'choice' column to the variable 'labels'.

In [ ]:
data = pd.read_csv("swarmIoT_full_labeled_dataset.csv")

labels = data['choice']

Replaces NaN values in the 'labels' variable with 0.

In [ ]:

nan_indices = np.where(np.isnan(labels))


nan_indices = nan_indices[0]


labels[nan_indices] = 0

Reads images from a specified directory using the cv2 library and appends them to a list called 'images'.

In [ ]:
images = []
path = "yourpathtotheimages"
for i in range(len(data)):
    image_name = data['image'][i]
    if '-' in image_name:
        image_name = image_name.split("-")[1]
    image_path = os.path.join(path, image_name)
    
    image = cv2.imread(image_path)
    images.append(image)

Converts the images in 'images' list to grayscale and appends them to a list called 'gray_images'. Grayscaling an image means converting it to a single-channel image, where each pixel is represented by a single intensity value, as opposed to a 3-channel image where each pixel is represented by 3 values (red, green, and blue).

In [ ]:
gray_images = []
for image in images:
    if image is not None:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray_images.append(gray)
    else:
        print("Skipping empty image:", data['image'][i])

Resizes the images in 'gray_images' to 100x100 and appends them to a list called 'resized_images'. Resizing the images is done to make the images all the same size, which is necessary for the next step of the code where the SIFT features are extracted. It's also possible that the model's input size needs to be fixed. The image size is resized to 100x100 pixels. This is an arbitrary size and can be changed depending on the requirements of the task and available computational resources.

In [1]:
resized_images = []
for gray in gray_images:
    resized = cv2.resize(gray, (100, 100))
    resized_images.append(resized)

NameError: name 'gray_images' is not defined

Extracts SIFT features from the images in 'resized_images' and appends them to a list called 'feature_vectors'.

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()
feature_vectors = []
for i in range(len(data)):
    try:
        keypoints, descriptor = sift.detectAndCompute(resized_images[i], None)
        feature_vectors.append(descriptor)
    except:
        print("Skipping image with no keypoints or descriptor:", data['image'][i])

valid_indices = [i for i in range(len(feature_vectors)) if feature_vectors[i].shape[1] == 128]
feature_vectors = [feature_vectors[i] for i in valid_indices]
labels = labels[valid_indices]

# reshape feature_vectors
feature_vectors = np.array(feature_vectors)
feature_vectors = feature_vectors.reshape(-1, feature_vectors.shape[-1])

Splits the data into training and test sets, using 80% of the data for training and 20% for testing.

In [1]:
X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.2, random_state=42)

NameError: name 'train_test_split' is not defined

Fits a RandomForestClassifier and a Support Vector Classifier (SVC) to the training data. ,both classifiers are trained on the same data and then used to predict the class of a new image. This allows for a comparison of the performance of the two classifiers on this particular dataset and task.

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)


clf1 = SVC()
clf1.fit(X_train, y_train)

Reads an image called "new_image.jpg", converts it to grayscale, resizes it to 100x100, extracts SIFT features from it, and predicts its class using the two trained classifiers. The predictions are printed to the console.

In [ ]:
print("predicting")
new_image = cv2.imread("new_image.jpg")
gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
resized = cv2.resize(gray, (100, 100))
keypoints, descriptor = sift.detectAndCompute(resized, None)
prediction = clf.predict(descriptor)
prediction1 = clf1.predict(descriptor)